Script to generate final UMAPs of interest.
-1st section: calculates and saves all UMAP reductions of interest in given csv file as well as all the graphs with the components of interest using the starting labeled sequences + IG data as inputs
-2nd section: calculates and plots the single specified UMAP using the starting labeled sequences + IG data as inputs
-3rd section: calculates and plots the single specified UMAP using a previously calculated UMAP reduction csv file as input


## Files to Import

In [ ]:
### importing mason data
### Substitute variabless for appropriate file locations

project_name = 'Mason'
# Name of project for file saving purposes

ig_data = r'/Users/isaacdaviet/Desktop/mason_igfold_models/masonIG.npy'
# numpy file containing one-hot-encoded intergraded gradients data

labeled_seqs= r'/Users/isaacdaviet/Desktop/mason_igfold_models/mason_sequences_label.csv'
# csv file containing sequences in column 1 and their status as binder (1)/non all (0) in the second column

save_folder = r'/Users/isaacdaviet/Desktop'
#save folder for graphs and cvs file.

## Setup

In [ ]:
import sys
sys.path.append('/Users/isaacdaviet/Desktop/thesis/python_versions')
# replace with directory containing the .py calculation files below
import umap_calc as umap
import pandas as pd

## Selected Parameters CSV UMAP Calculation

In [ ]:
selected_parameters_csv = r'/Users/isaacdaviet/Desktop/graphs/UMAP_analysis/UMAP_final_parameters_test.csv'
# csv file containing final parameter of interest to test/plot. column format: [umap_calc, distance metric, n_components, n_neighbors, min_dist, component_1, component_2, component_3]. INCLUDE THESE EXACT COLUMN NAMES

points_to_show = ['all_pts', 'binders', 'non_binders']
# what points to show on graphs. 'all' shows binders and non binders in same graph, 'binders' show only binders, 'non binders' shows only non binders

save_graph = 'n' 
# Switch toye to save resulting graph in save folder

umap.umap_final_plts_csv(ig_data, labeled_seqs, selected_parameters_csv,
                         points_to_show = points_to_show,
                         plt_size = [60, 20], # final plot size
                         pt_size= 4, # point size
                         fontsize=64, #title font size
                         decimanl_axes = 'n', # 'y' to show decimal point axes, 'n' to not
                         save_graph = save_graph, 
                         save_folder = save_folder,
                         project_name=project_name,
                         save_3d_graph = 'n') # If 3 components are given, will generate an explorable 3D plotly graph 

## Individual UMAP Calculation and Plotting

#### From Starting Labeled Sequence + IG Data files

In [ ]:
# Reduction Calculation Parameters
metric='hamming'
n_components=3
n_neighbors=25
min_dist=0.1

# Plotting Parameters
plot_graph = 'y' # Set to 'y' to plot graph after UMAP reduction calculated
component1=2 # x_axis component
component2=3 # y_axis component
show_pts='Binder' # filter selection. Select 'all', 'Binder', or 'Non Binder' to display only those points
iterate_all_pt_types = 'n' # If set to 'y' and show_pts = 'all', will generate graphs for all points, binders only & non-binder only


# CSV Saving parameters
save_results_csv = 'n' # Change to 'y' to save reduction as csv
csv_save_folder = None
csv_file_name = None # include '.csv' file extension

show_graph='y' # set to 'y' to show graph in console
save_graph=None # set to 'y' to save graphs
graph_save_folder = None
graph_file_name = None # include .png/.jpeg file extension

In [ ]:
ig_df = umap.flatten_from_files(ig_data, labeled_seqs)

ohe_df = ig_df.applymap(lambda x:1 if x!=0 else 0)

ohe_df['Labels'] = ohe_df['Labels'].apply(lambda x: 'Binder' if x == 1 else 'Non Binder')

ohe_df =ohe_df.drop(columns=['Sequences'])

ohe_umap = umap.umap_reduction(ohe_df, n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist, metric=metric, random_state=42, save_results_csv = save_results_csv, save_folder = csv_save_folder, file_name = csv_file_name)

if plot_graph == 'y':
    graph_save_path = f"{graph_save_folder}/{graph_file_name}" if None not in [graph_save_folder, graph_file_name] else None

    umap.plot_umap(ohe_umap, component1=component1, component2=component2, metric=metric, n_neighbors = n_neighbors, min_dist = min_dist, show_pts=show_pts, show_graph=show_graph, save_graph=save_graph, save_path=graph_save_path, project_name=project_name, iterate_all_pt_types = iterate_all_pt_types, close_plot = 'y')

#### Plot From Previously Calculated UMAP Reduction

In [ ]:
reduction_file_path = '/Users/isaacdaviet/Desktop/results/Mason_umap_final_graphs/csv_files/Mason-FinalSelect_UMAP-correlation_nC3_Ne25_MD0.15.csv'
# CSV file of previously calculated reduction to be plotted

### Parameters of UMAP Reduction for Plot Title
# Reduction Calculation Parameters
metric='hamming'
n_components=3
n_neighbors=25
min_dist=0.1

# Plotting Parameters
component1=2 # x_axis component
component2=3 # y_axis component
show_pts='Binder' # filter selection. Select 'all', 'Binder', or 'Non Binder' to display only those points
iterate_all_pt_types = 'n' # If set to 'y' and show_pts = 'all', will generate graphs for all points, binders only & non-binder only

show_graph='y' # set to 'y' to show graph in console
save_graph=None # set to 'y' to save graphs
graph_save_folder = None
graph_file_name = None # include .png/.jpeg file extension



reduction_df = pd.read_csv(reduction_file_path)

ohe_umap['Labels'] = reduction_df['Labels']
graph_save_path = f"{graph_save_folder}/{graph_file_name}" if None not in [graph_save_folder, graph_file_name] else None

umap.plot_umap(ohe_umap, component1=component1, component2=component2, metric=metric, n_neighbors = n_neighbors, min_dist = min_dist, show_pts=show_pts, show_graph=show_graph, save_graph=save_graph, save_path=graph_save_path, project_name=project_name, iterate_all_pt_types = iterate_all_pt_types, close_plot = 'y')